# Working with BIDS data

In [ ]:
import matplotlib
import pathlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')

## Read the input data we wish to convert to BIDS

In [ ]:
sample_data_dir = mne.datasets.sample.data_path()
sample_data_dir = pathlib.Path(sample_data_dir)

raw_path = sample_data_dir / 'MEG' / 'sample' / 'sample_audvis_raw.fif'
raw = mne.io.read_raw(raw_path)

events = mne.find_events(raw)
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}

## Write the raw data to BIDS!

We need to specify the power line frequency.

In [ ]:
raw.info['line_freq'] = 60

Now, actually write the data.

In [ ]:
out_path = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              root=out_path)

mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events,
                        event_id=event_id, overwrite=True)

## Write MEGIN / Elekta / NeuroMag fine-calibration and crosstalk files

In [ ]:
cal_path = sample_data_dir / 'SSS' / 'sss_cal_mgh.dat'
ct_path = sample_data_dir / 'SSS' / 'ct_sparse_mgh.fif'

mne_bids.write_meg_calibration(cal_path, bids_path=bids_path)
mne_bids.write_meg_crosstalk(ct_path, bids_path=bids_path)


<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Write the same raw data again, but this time act as if it were from a different subject – subject "02".</li>
    </ul>
</div>

## Print the created file & directory structure

In [ ]:
mne_bids.print_dir_tree(out_path)

## Generate a data summary

In [ ]:
print(mne_bids.make_report(out_path))

## Reading BIDS data

Let's try to read back the data we just stored according to the BIDS specification.

In [ ]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)

In [ ]:
raw.plot()

## Events are stored as annotations – but we convert between the two.

In [ ]:
raw.annotations[0]

In [ ]:
events, event_id = mne.events_from_annotations(raw)

In [ ]:
mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])

## Fine the MEG fine-calibration and crosstalk files

In [ ]:
bids_path.meg_calibration_fpath

In [ ]:
bids_path.meg_crosstalk_fpath